In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/19 14:24:14 WARN Utils: Your hostname, xieyufengs-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.13.50.12 instead (on interface en0)
22/09/19 14:24:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 14:24:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df = pd.read_parquet('../data/curated/mechant_consumer_abs')
df

In [25]:
df_agg = df.groupby(by=['merchant_name']).agg(
    total_transaction=('postcode', 'count'),
    total_revenue=('dollar_value', np.sum),
    ).reset_index()


In [26]:
df_agg

,merchant_name,total_transaction,total_revenue
0,A Aliquet Ltd,923,195789.626317
1,A Arcu Industries,2272,834906.529502
2,A Arcu Sed Company,1155,150267.314855
3,A Arcu Sed Corporation,18,10786.692787
4,A Associates,68,24638.615042
...,...,...,...
3886,Vulputate Ullamcorper Magna LLC,89,33455.109989
3887,Vulputate Ullamcorper Magna Ltd,74,26325.078990
3888,Vulputate Velit Eu Incorporated,44,20997.614845
3889,Vulputate Velit Eu Limited,11871,229909.434371


In [30]:
df2 = df.groupby(by=['merchant_name', 'order_year', 'order_month']).agg(
    revenue=('dollar_value', np.sum),
    ).reset_index()
df2

,merchant_name,order_year,order_month,revenue
0,A Aliquet Ltd,2021,2,248.226823
1,A Aliquet Ltd,2021,3,9032.258746
2,A Aliquet Ltd,2021,4,8735.797479
3,A Aliquet Ltd,2021,5,6258.212153
4,A Aliquet Ltd,2021,6,9304.704191
...,...,...,...,...
71921,Vulputate Velit Inc.,2022,6,18507.792327
71922,Vulputate Velit Inc.,2022,7,22429.588509
71923,Vulputate Velit Inc.,2022,8,21535.642306
71924,Vulputate Velit Inc.,2022,9,27785.127512


In [28]:
TIME_SPAN = int(365*1.5)
df_agg['transaction_per_day'] = df_agg['total_transaction'] / TIME_SPAN
df_agg['revenue_per_day'] = df_agg['total_revenue'] * df_agg['transaction_per_day']
df_agg

,merchant_name,total_transaction,total_revenue,transaction_per_day,revenue_per_day
0,A Aliquet Ltd,923,195789.626317,1.687386,3.303726e+05
1,A Arcu Industries,2272,834906.529502,4.153565,3.467838e+06
2,A Arcu Sed Company,1155,150267.314855,2.111517,3.172920e+05
3,A Arcu Sed Corporation,18,10786.692787,0.032907,3.549552e+02
4,A Associates,68,24638.615042,0.124314,3.062936e+03
...,...,...,...,...,...
3886,Vulputate Ullamcorper Magna LLC,89,33455.109989,0.162706,5.443336e+03
3887,Vulputate Ullamcorper Magna Ltd,74,26325.078990,0.135283,3.561345e+03
3888,Vulputate Velit Eu Incorporated,44,20997.614845,0.080439,1.689022e+03
3889,Vulputate Velit Eu Limited,11871,229909.434371,21.702011,4.989497e+06
